In [5]:
#IMPORTANDO LIBRERIAS
import pandas as pd
import numpy as np
import random as rn
import time
import datetime
import logging
import warnings
import os
!pip install unidecode
from unidecode import unidecode as u
import pathlib
from pathlib import Path

#CARGAR BASE DE DATOS MATERIAS-SEMESTRES-CRÉDITOS. ESTUDIANTES
dfestudiantes = pd.read_csv('https://raw.githubusercontent.com/Danielavelz/TRABAJO-FINAL/refs/heads/main/estudiantes.csv',encoding='ISO-8859-1', delimiter= ';')
dfmaterias = pd.read_csv('https://raw.githubusercontent.com/Danielavelz/TRABAJO-FINAL/refs/heads/main/MATERIAS%20(1).csv',encoding='ISO-8859-1', delimiter= ';')

# Revisar las columnas disponibles
print(dfestudiantes.columns)

# Ordenar nombres de columnas
dfestudiantes.columns = dfestudiantes.columns.str.strip().str.lower().str.replace(' ', '_')

def GenerearSemestre():
    semestre = []
    for i in range(1, 11):
        if i == 1:
            cantidad = 1000
        elif i == 2:
            cantidad = 700
        elif i == 3:
            cantidad = 490
        elif i == 4:
            cantidad = 343
        elif i == 5:
            cantidad = 240
        elif i == 6:
            cantidad = 168
        elif i == 7:
            cantidad = 117
        elif i == 8:
            cantidad = 82
        elif i == 9:
            cantidad = 57
        else:
            cantidad = 40
        semestre.extend([i] * cantidad)
    rn.shuffle(semestre)
    return semestre

# Función para generar nota definitiva
def generar_nota_definitiva():
    if rn.random() < 0.7:  # 70% de probabilidad de aprobar
        nota_definitiva = round(rn.uniform(3.0, 5.0), 1)
    else:
        nota_definitiva = round(rn.uniform(0.0, 2.9), 1)
    estado = "APROBADO" if nota_definitiva >= 3.0 else "REPROBADO"
    return nota_definitiva, estado

# Nombramos la Nueva función para asignar grupos de curso
def asignar_grupo_cc(row):
    """
    Asigna un código de curso (CC) a un estudiante basado en su semestre y asignatura.
    """
    cursos_disponibles = cursos[
        (cursos['ASIGNATURA'] == row['ASIGNATURA']) &
        (cursos['SEMESTRE'] == row['SEMESTRE'])
    ]['CC'].tolist()

    if cursos_disponibles:
        return rn.choice(cursos_disponibles)
    else:
        return None

semestres_generados = GenerearSemestre()
df2 = pd.DataFrame({'SEMESTRE': semestres_generados})

# Concatenar nombre y apellido con el semestre
Alumnos = pd.concat([dfestudiantes, df2], axis=1)

# Cargar el archivo desde Github
asignaturas = pd.read_csv("https://raw.githubusercontent.com/Danielavelz/TRABAJO-FINAL/main/materias%20sin%20cod.csv", encoding="latin1")

# Eliminar la columna "NUCLEO_CURRICULAR"
asignaturas = asignaturas.drop(columns=["NUCLEO_CURRICULAR"])

# Generar código para cada asignatura (CA)
def generar_numero_aleatorio(unicos):
    while True:
        numero = rn.randint(0, 9)
        if numero not in unicos:
            unicos.add(numero)
            return numero

def generar_codigo(materia, codigos_existentes, unicos):
    # Tomar hasta 3 palabras del nombre de la asignatura
    palabras = materia['ASIGNATURA'].split()[:3]
    # Tomar la primera letra de cada palabra y asegurar que tengamos 3 caracteres
    iniciales = ''.join(palabra[0].upper() for palabra in palabras)
    # Si tenemos menos de 3 caracteres, completar con la siguiente letra de la primera palabra
    while len(iniciales) < 3:
        palabras_idx = len(iniciales) - len(palabras[0])
        if palabras_idx < len(palabras[0]):
            iniciales += palabras[0][palabras_idx].upper()
        else:
            iniciales += 'X'

    semestre = str(materia['SEMESTRE'])
    creditos = str(materia['CREDITOS'])
    while True:
        numero_aleatorio = str(rn.randint(0, 9))
        # Crear código de 6 caracteres: 3 de iniciales + 1 semestre + 1 créditos + 1 aleatorio
        CA = iniciales + semestre + creditos + numero_aleatorio
        if CA not in codigos_existentes:
            codigos_existentes.add(CA)
            return CA

codigos_existentes = set()
asignaturas['CA'] = asignaturas.apply(lambda materia: generar_codigo(materia, codigos_existentes, set()), axis=1)

def generar_aulas(SEMESTRE):
    if SEMESTRE == 1:
        return 1000/40
    elif SEMESTRE == 2:
        return 700/40
    elif SEMESTRE == 3:
        return 490/40
    elif SEMESTRE == 4:
        return 343/35
    elif SEMESTRE == 5:
        return 240/35
    elif SEMESTRE == 6:
        return 168/35
    elif SEMESTRE == 7:
        return 117/25
    elif SEMESTRE == 8:
        return 82/25
    elif SEMESTRE == 9:
        return 57/25
    else:
        return 40/10

asignaturas['TCA'] = asignaturas['SEMESTRE'].apply(generar_aulas)
asignaturas['TCA'] = asignaturas['TCA'].round(2)

def generar_cant_estudiantes(SEMESTRE):
    if SEMESTRE == 1:
        return 1000
    elif SEMESTRE == 2:
        return 700
    elif SEMESTRE == 3:
        return 490
    elif SEMESTRE == 4:
        return 343
    elif SEMESTRE == 5:
        return 240
    elif SEMESTRE == 6:
        return 168
    elif SEMESTRE == 7:
        return 117
    elif SEMESTRE == 8:
        return 82
    elif SEMESTRE == 9:
        return 57
    else:
        return 40

asignaturas['NTE'] = asignaturas['SEMESTRE'].apply(generar_cant_estudiantes)

# Nuevo DataFrame con todos los cursos creados para cada asignatura
codigos_repetidos = asignaturas.index.repeat(asignaturas["TCA"])
cursos = asignaturas.loc[codigos_repetidos].reset_index(drop=True)

# Generar código de los cursos (CC)
fecha_actual = datetime.datetime.now().strftime('%Y-%m-%d')
cursos["CC"] = range(1, len(cursos) + 1)
cursos["FC"] = fecha_actual

# Realizar el merge para combinar los dataframes
ESTUDIANTES = pd.merge(Alumnos, asignaturas, on='SEMESTRE')

# Asignar CC a cada estudiante
ESTUDIANTES['CC'] = ESTUDIANTES.apply(asignar_grupo_cc, axis=1)

# Eliminar columnas innecesarias
columnas_a_eliminar = ['CAP-MAX', 'TCA']
ESTUDIANTES = ESTUDIANTES.drop(columns=[col for col in columnas_a_eliminar if col in ESTUDIANTES.columns])

# Generar notas definitivas para cada estudiante
notas_estudiantes = []
for _, estudiante in ESTUDIANTES.iterrows():
    nota_definitiva, estado = generar_nota_definitiva()
    notas_estudiantes.append({
        'NOTA_DEFINITIVA': nota_definitiva,
        'ESTADO': estado
    })

# Convertir las notas a DataFrame y concatenar con ESTUDIANTES
df_notas = pd.DataFrame(notas_estudiantes)
ESTUDIANTES = pd.concat([ESTUDIANTES, df_notas], axis=1)

# Crear estructura de carpetas y guardar archivos
base_path = 'Ruta Trabajo Final'
for semestre in asignaturas['SEMESTRE'].unique():
    semestre_path = os.path.join(base_path, f'semestre {semestre}')
    os.makedirs(semestre_path, exist_ok=True)

    for asignatura in asignaturas[asignaturas['SEMESTRE'] == semestre]['ASIGNATURA']:
        asignatura_path = os.path.join(semestre_path, asignatura)
        os.makedirs(asignatura_path, exist_ok=True)

        # Filtrar estudiantes por semestre y asignatura
        Alumnos = ESTUDIANTES[ESTUDIANTES['SEMESTRE'] == semestre]
        Alumnos_semestre = Alumnos[Alumnos["ASIGNATURA"] == asignatura]

        # Guardar archivos
        archivo_excel = os.path.join(asignatura_path, 'Estudiantes.xlsx')
        archivo_csv = os.path.join(asignatura_path, 'Estudiantes.csv')
        Alumnos_semestre.to_excel(archivo_excel, index=False)
        Alumnos_semestre.to_csv(archivo_csv, index=False)

# Crear un resumen estadístico de las notas
def generar_estadisticas_notas():
    estadisticas = ESTUDIANTES.groupby(['SEMESTRE', 'ASIGNATURA']).agg({
        'NOTA_DEFINITIVA': ['mean', 'min', 'max', 'std'],
        'ESTADO': lambda x: (x == 'APROBADO').mean() * 100
    }).round(2)
    estadisticas.columns = ['Promedio', 'Mínima', 'Máxima', 'Desv. Estándar', '% Aprobación']
    return estadisticas

# Guardar promedios
estadisticas = generar_estadisticas_notas()
estadisticas.to_excel('Estadisticas_Notas.xlsx')
estadisticas.to_csv('Estadisticas_Notas.csv')

Index(['Nombre completo '], dtype='object')
